In [1]:
import json
import pandas as pd

In [2]:
json_path = r"C:\Users\Nidhi\OneDrive\Documents\Sem VII\GEN_AI\data\detections_with_gps.json"

In [3]:
with open(json_path, "r") as f:
    data = json.load(f)

In [4]:
records = []
for item in data:
    lat = item["location"].get("latitude")
    lon = item["location"].get("longitude")
    cls = item["detections"][0]["class"] if item["detections"] else None
    img_path = item["image_path"]
    
    # Assign municipal authority based on coordinates
    if lat is not None and lon is not None:
        if 19.0 <= lat <= 19.2 and 72.8 <= lon <= 72.9:
            authority = "BMC (Mumbai)"
        elif 18.95 <= lat <= 19.0 and 73.1 <= lon <= 73.2:
            authority = "NMCC (Navi Mumbai)"
        elif 19.18 <= lat <= 19.25 and 73.0 <= lon <= 73.05:
            authority = "Thane Municipal Corporation"    
        else:
            authority = "Local Authority"
    else:
        authority = "Unknown"

    records.append([img_path, cls, lat, lon, item["timestamp"], authority])

df = pd.DataFrame(records, columns=["Image", "Class", "Latitude", "Longitude", "Timestamp", "Authority"])
df.head()

,Image,Class,Latitude,Longitude,Timestamp,Authority
0,C:\Users\Mahika\Desktop\cv_gai\data\test\image...,not-pothole,19.1130,72.8690,2025-10-26T08:23:12.578624,BMC (Mumbai)
1,C:\Users\Mahika\Desktop\cv_gai\data\test\image...,potholes,19.1130,72.8690,2025-10-26T08:23:12.601411,BMC (Mumbai)
2,C:\Users\Mahika\Desktop\cv_gai\data\test\image...,not-pothole,18.9897,73.1160,2025-10-26T08:23:12.658358,NMCC (Navi Mumbai)
3,C:\Users\Mahika\Desktop\cv_gai\data\test\image...,potholes,19.1196,72.8460,2025-10-26T08:23:12.684919,BMC (Mumbai)
4,C:\Users\Mahika\Desktop\cv_gai\data\test\image...,potholes,19.2173,73.0936,2025-10-26T08:23:12.700935,Local Authority


In [5]:
import os
from dotenv import load_dotenv


In [ ]:

import google.generativeai as genai




In [ ]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [ ]:


genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
import google.generativeai as genai

# Load API key
load_dotenv(dotenv_path=r"C:\Users\Nidhi\OneDrive\Documents\Sem VII\GEN_AI\data\.env")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# List available models
models = genai.list_models()
for model in models:
    print(f"Model Name: {model.name}")


Model Name: models/embedding-gecko-001
Model Name: models/gemini-2.5-pro-preview-03-25
Model Name: models/gemini-2.5-flash-preview-05-20
Model Name: models/gemini-2.5-flash
Model Name: models/gemini-2.5-flash-lite-preview-06-17
Model Name: models/gemini-2.5-pro-preview-05-06
Model Name: models/gemini-2.5-pro-preview-06-05
Model Name: models/gemini-2.5-pro
Model Name: models/gemini-2.0-flash-exp
Model Name: models/gemini-2.0-flash
Model Name: models/gemini-2.0-flash-001
Model Name: models/gemini-2.0-flash-exp-image-generation
Model Name: models/gemini-2.0-flash-lite-001
Model Name: models/gemini-2.0-flash-lite
Model Name: models/gemini-2.0-flash-preview-image-generation
Model Name: models/gemini-2.0-flash-lite-preview-02-05
Model Name: models/gemini-2.0-flash-lite-preview
Model Name: models/gemini-2.0-pro-exp
Model Name: models/gemini-2.0-pro-exp-02-05
Model Name: models/gemini-exp-1206
Model Name: models/gemini-2.0-flash-thinking-exp-01-21
Model Name: models/gemini-2.0-flash-thinking-e

In [ ]:
from dotenv import load_dotenv
import google.generativeai as genai


load_dotenv(dotenv_path=r"C:\Users\Nidhi\OneDrive\Documents\Sem VII\GEN_AI\data\.env")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


def get_prompt(row):
    return f"""
Write a formal complaint letter addressed to the {row['Authority']}.
The pothole was detected at Latitude: {row['Latitude']}, Longitude: {row['Longitude']}.
Mention that it causes inconvenience to commuters and should be repaired soon.
End with a polite closing.
"""

# ==========================
# 5. Google Gemini Letter Generation (correct API)
# ==========================
def generate_with_google(row, model_name="models/gemini-2.5-flash"):
    prompt = get_prompt(row)
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(prompt)
    return response.text.strip()

# ==========================
# 6. Letter Generation Pipeline
# ==========================
output_dir = "generated_letters"
os.makedirs(output_dir, exist_ok=True)

for idx, row in df.iterrows():
    if row["Class"] == "potholes":
        print(f"\n🕳 Generating letter for {row['Authority']} at ({row['Latitude']}, {row['Longitude']})")
        try:
            letter = generate_with_google(row)

            letter_path = os.path.join(output_dir, f"letter_{idx+1}.txt")
            with open(letter_path, "w", encoding="utf-8") as f:
                f.write(f"===== GOOGLE LETTER ({row['Authority']}) =====\n")
                f.write(letter)

            print(f"✅ Letter saved: {letter_path}")

        except Exception as e:
            print(f"⚠️ Error generating letter for row {idx+1}: {e}")

print("\n🎉 All pothole complaint letters generated successfully!")



🕳 Generating letter for BMC (Mumbai) at (19.113, 72.869)
✅ Letter saved: generated_letters\letter_2.txt

🕳 Generating letter for BMC (Mumbai) at (19.1196, 72.846)
✅ Letter saved: generated_letters\letter_4.txt

🕳 Generating letter for Local Authority at (19.2173, 73.0936)
✅ Letter saved: generated_letters\letter_5.txt

🎉 All pothole complaint letters generated successfully!


In [44]:
pip install langchain google-generativeai python-dotenv pandas



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
pip install -U langchain-google-genai


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 9.1 MB/s eta 0:00:00

  Attempting uninstall: google-ai-generativelanguage

    Found existing installation: google-ai-generativelanguage 0.6.15

   ------------- -------------------------- 1/3 [google-ai-generativelanguage]
   ------------- -------------------------- 1/3 [google-ai-generativelanguage]
   ------------- -------------------------- 1/3 [google-ai-generativelanguage]
    Uninstalling google-ai-generativelanguage-0.6.15:
   ------------- -------------------------- 1/3 [google-ai-generativelanguage]
      Successfully uninstalled google-ai-generativelanguage-0.6.15
   ------------- -------------------------- 1/3 [google-ai-generativelanguage]
   ------------- -------------------------- 1/3 [google-ai-generativelanguage]
   ------------- -------------------------- 1/3 [google-ai-generativelanguage]
   ------------- -------------------------- 1/3 [google

In [ ]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# ==========================
# 1. Load environment variables
# ==========================
load_dotenv(dotenv_path=r"C:\Users\Nidhi\OneDrive\Documents\Sem VII\GEN_AI\data\.env")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# ==========================
# 2. Initialize LangChain Google Gemini chat model
# ==========================
chat_model = ChatGoogleGenerativeAI(
    model="models/gemini-2.5-flash",  # or "models/gemini-2.5-pro"
    api_key=GOOGLE_API_KEY,
    temperature=0.7
)


# ==========================
# 5. Prompt Template
# ==========================
def get_prompt(row):
    return f"""
Write a formal complaint letter addressed to the {row['Authority']}.
The pothole was detected at Latitude: {row['Latitude']}, Longitude: {row['Longitude']}.
Mention that it causes inconvenience to commuters and should be repaired soon.
End with a polite closing.
"""

# ==========================
# 6. Generate letters
# ==========================
output_dir = "generated_letters_langchain"
os.makedirs(output_dir, exist_ok=True)

for idx, row in df.iterrows():
    if row["Class"] == "potholes":
        print(f"\n🕳 Generating letter for {row['Authority']} at ({row['Latitude']}, {row['Longitude']})")
        try:
            prompt = get_prompt(row)
            response = chat_model.invoke(prompt)  # just pass string
            letter = response.content.strip()

            letter_path = os.path.join(output_dir, f"letter_{idx+1}.txt")
            with open(letter_path, "w", encoding="utf-8") as f:
                f.write(f"===== GOOGLE GEMINI LETTER ({row['Authority']}) =====\n")
                f.write(letter)

            print(f"✅ Letter saved: {letter_path}")

        except Exception as e:
            print(f"⚠️ Error generating letter for row {idx+1}: {e}")

print("\n🎉 All pothole complaint letters generated successfully!")



🕳 Generating letter for BMC (Mumbai) at (19.113, 72.869)
✅ Letter saved: generated_letters_langchain\letter_2.txt

🕳 Generating letter for BMC (Mumbai) at (19.1196, 72.846)
✅ Letter saved: generated_letters_langchain\letter_4.txt

🕳 Generating letter for Local Authority at (19.2173, 73.0936)
✅ Letter saved: generated_letters_langchain\letter_5.txt

🎉 All pothole complaint letters generated successfully!


In [ ]:

def get_prompt(row):
    return f"""
Write a formal complaint letter addressed to the {row['Authority']}.
The pothole was detected at Latitude: {row['Latitude']}, Longitude: {row['Longitude']}.
Mention that it causes inconvenience to commuters and should be repaired soon.
End with a polite closing.
"""
 
# ==========================
# 5. Letter Generation using Ollama
# ==========================
output_dir = "E:\Gen_ai\generated_letters_ollama"
os.makedirs(output_dir, exist_ok=True)
 
for idx, row in df.iterrows():
    if row["Class"] == "potholes":
        print(f"\n🕳 Generating letter for {row['Authority']} at ({row['Latitude']}, {row['Longitude']})")
        try:
            prompt = get_prompt(row)
            # Generate with local model
            response = ollama.chat(model="llama3.1:8b", messages=[{"role": "user", "content": prompt}])
            letter = response['message']['content'].strip()
 
            # Save letter
            letter_path = os.path.join(output_dir, f"letter_{idx+1}.txt")
            with open(letter_path, "w", encoding="utf-8") as f:
                f.write(f"===== OLLAMA LETTER ({row['Authority']}) =====\n")
                f.write(letter)
 
            print(f"✅ Letter saved: {letter_path}")
 
        except Exception as e:
            print(f"⚠️ Error generating letter for row {idx+1}: {e}")
 
print("\n🎉 All pothole complaint letters generated successfully using Ollama!")


🕳 Generating letter for BMC (Mumbai) at (19.113, 72.869)
✅ Letter saved: E:\Gen_ai\generated_letters_ollama\letter_2.txt

🕳 Generating letter for BMC (Mumbai) at (19.1196, 72.846)
✅ Letter saved: E:\Gen_ai\generated_letters_ollama\letter_4.txt

🕳 Generating letter for Local Authority at (19.2173, 73.0936)
✅ Letter saved: E:\Gen_ai\generated_letters_ollama\letter_5.txt

🎉 All pothole complaint letters generated successfully using Ollama!


: 